### Pengecekaan 

In [1]:
import pandas as pd
import concurrent.futures
import time
import re

from nltk.tokenize import word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer

# Initialize Sastrawi stopword remover
stopword_factory = StopWordRemoverFactory()
stopword_remover = stopword_factory.create_stop_word_remover()

# Initialize Sastrawi stemmer
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

# Porter
porter_stemmer = PorterStemmer()

cleaning

In [2]:
teks1='"Panglima TNI Laksamana Yudo Margono buka suara soal sosok KSAD baru Jenderal TNI Agus Subiyanto yang disodorkan Presiden RI untuk dites DPR sebagai calon penggantinya.Yudo yang akan pensiun itu meyakini kemampuan Agus seandainya resmi memimpin TNI sebagai pengganti dirinya kelak.""Pak Agus Subiyanto baik. Saya yakin beliau mampu untuk memimpin TNI, tentunya nanti bersinergi dengan Polri dan juga bisa bersama sama kepala staf yang lain untuk membawa TNI yang lebih baik, lebih profesional, modern, tangguh,"" kata Yudo di Sesko TNI, Bandung, Kamis (2/11).Yudo menuturkan saat dirinya dilantik, ada beberapa agenda kegiatan besar TNI yang telah disusunnya sebagai panglima. Salah satunya adalah mengawal proses berjalan Pemilu 2024. Itu, kata dia, akan menjadi pijakan juga bagi panglima pengganti dirinya.ADVERTISEMENT""Dalam perencanaan kegiatan sudah ada semuanya, yang kita susun di tahun 2023 ini dan nantinya 2024 apa saja, tentunya Pemilu, berikutnya dalam operasi latihan, operasi juga tetap sekarang yang ada, latihan juga kita rancang di tahun 2024 nanti,"" kata Yudo yang akan memasuki masa pensiun pada akhir November 2023 ini.DPR telah menerima Surat Presiden (Surpres) dari Jokowi untuk menguji kepatutan dan kelayakan (fit and proper test) Agus Subiyanto sebagai calon Panglima TNI. Komisi I DPR pun telah menjadwalkan uji kepatutan dan kelayakan itu bakal digelar pada 14 November mendatang.Yudo sendiri, saat disinggung langkah selanjutnya setelah melepas jabatan dan purnawirawan, mengaku ingin memfokuskan diri bertani.""Sudah saya jawab, bertani. Bertani kan banyak, ada bertani padi, kebun, banyak. Bertani lah pokoknya. (Tempat bertaninya) Rahasia,"" ujar Yudo."'
teks2='"KOMPAS.com - Gubernur Jawa Timur (Jatim) Khofifah Indar Parawansa mengaku bangga dengan capaian indeks pembangunan manusia (IPM) Jatim yang terus membaik hingga akhir 2023.Badan Pusat Statistik (BPS) menyebutkan, IPM Jatim mencapai 74,65 per 1 Desember 2023, atau meningkat 3,15 persen selama periode 2019-2023.Capaian IPM Jatim 2023 tersebut di atas rata-rata nasional sebesar 74,39 serta provinsi besar lain di Pulau Jawa, yaitu Jawa Barat 74,24 dan Jawa Tengah sebesar 73,39.Khofifah pun optimistis Jatim akan mampu mewujudkan target sasaran visi Indonesia Emas sesuai Rencana Pembangunan Jangka Menengah Nasional (RPJMN) 2025-2045.""Dengan capaian IPM Jatim ini, kami semakin optimis Jatim akan mampu mewujudkan Indonesia Emas 2045,"" terangnya di Gedung Negara Grahadi Surabaya, Senin (4/12/2023).Baca juga: Pakar UB Dorong Pemprov Jatim Jadi Pionir Penerapan Kurikulum KebencanaanTerkait capaian itu, Khofifah mengatakan, Pemerintah Provinsi (Pemprov) Jatim menyiapkan strategi pembangunan untuk mendukung peningkatan IPM.Dia mencontohkan, pihaknya menjalankan strategi kesetaraan vokasional berupa program Kejar Paket A, B, dan C Vokasi, penyediaan Pendidikan Gratis dan Berkualitas (Tis-Tas), hingga Bantuan Operasional Sekolah Daerah untuk Madrasah Diniah (Bosda Madin).Khofifah mengatakan, strategi ini tidak hanya meningkatkan rata-rata lama sekolah, tetapi juga keterampilan bagi peserta kejar paket dengan menambahkan muatan vokasional.?Dengan begitu, mereka siap secara skill untuk bekerja dan berusaha yang berdampak pula pada meningkatnya daya beli,"" ujarnya dalam siaran pers.Mantan Menteri Sosial (Mensos) itu mengatakan, upaya Pemprov Jatim meningkatkan prestasi akademik dan non akademik di bidang pendidikan juga membanggakan.Baca juga: Pakar UB Dorong Pemprov Jatim Jadi Pionir Penerapan Kurikulum KebencanaanHal itu terlihat dari capaian sebagai juara umum tiga tahun berturut-turut Olimpiade Sains Nasional (OSN) periode tahun 2020?2022 dan Juara Umum Lomba Kompetensi Siswa (LKS) SMK tingkat Nasional 2023.Jatim juga menjadi provinsi dengan jumlah mahasiswa terbanyak yang diterima Seleksi Nasional Masuk Perguruan Tinggi Negeri (SNMPTN) dan Seleksi Bersama Masuk Perguruan Tinggi Negeri (SBMPTN) selama empat tahun berturut-turut, yaitu pada 2020-2023.Pemprov Jatim juga memiliki strategi peningkatan IPM atau Human Development Index (HDI) terkait umur harapan hidup (UHH) dan kesehatan.Beberapa strategi tersebut, seperti Pondok Kesehatan Desa (Ponkesdes), Puskesmas Rawat Inap Standar, Taman Posyandu, Pendampingan Bumil Risti, Pencegahan Stunting bagi Ibu Hamil, hingga Tantistas (Kesehatan Gratis dan berkualitas).Strategi Pemprov Jatim lainnya adalah peningkatan IPM untuk standar hidup layak.Baca juga: Khofifah Sebut UMK Jatim 2024 Adil bagi Pekerja dan Pengusaha"'
teks3='"Ketua DPP PKB Ahmad Iman Sukri mengklaim elektabilitas Ketua Umum PKB Muhaimin Iskandar (Cak Imin) sebagai bakal calon wakil presiden unggul dari Gubernur Jawa Timur Khofifah Indar Parawansa dan Menko Polhukam Mahfud MD di wilayah Jawa Timur.Ia mengatakan survei itu dilakukan secara internal oleh PKB melalui lembaga Polmark sekitar lima bulan yang lalu.""Kita lakukan survei seluruh Indonesia, responden 70 ribu. Nah, hasil survei di Jatim cawapres tertinggi itu Cak Imin. Itu Polmark. Kurang lebih 12-14 persen. Saya lupa. Baru Khofifah, ada Mahfud, dan lain-lain,"" kata Iman dalam podcast What The Fact! Politics CNN Indonesia yang tayang pada Jumat (29/9).Selain di Jatim, Iman mengidentifikasi pasangan Anies Baswedan dan Cak Imin akan unggul di Jakarta, Sumatera, beberapa wilayah Kalimantan, hingga di Sulawesi.ADVERTISEMENTIa meyakini Ketua Umum NasDem Surya Paloh dan Anies memilih Cak Imin sebagai cawapres Anies dengan pertimbangan rasional.""Karena itu berdasarkan data survei juga,"" kata dia.Iman juga optimistis referensi politik warga Nahdlatul Ulama (NU) di Jatim akan mengerucut ke PKB. Ia mengklaim selama ini konstituen PKB merupakan warga NU.Karena itu, ia pun yakin konsolidasi pasangan Anies-Cak Imin makin menggumpal di Jatim untuk meraih kemenangan di Pilpres 2024. Ia mengungkapkan nantinya Anies dan Cak Imin makin sering keliling di Jawa Timur untuk sosialisasi.""Meski sekarang survei kecil, tapi kita tak berkecil hati,"" kata Iman.""Nanti kita buat acara di Sidoarjo. Kita konsolidasi dan panaskan mesin,"" tambahnya.Selain itu, Iman menghargai para bakal calon presiden lainnya kini berencana menggandeng tokoh dari NU sebagai cawapres. Menurut dia, kondisi ini membuat semangat pertarungan pilpres menjadi bergairah dan lebih seru.""Cak Imin itu jadi cawapres dari kalangan NU itu hidup lagi kartunya. Kalau Cak Imin tak deklarasi ke Anies, kartu Mahfud dan Khofifah belum tentu hidup lagi kan. Tetap kita yakin pertarungan akan seru,"" kata dia."'

teks_array = []

content_cleaned = re.sub(r'ADVERTISEMENT', '', teks1)
content_cleaned = re.sub(r'KOMPAS.com', '', content_cleaned)
content_cleaned = re.sub(r'[^a-zA-Z0-9\s]+', ' ', content_cleaned)
teks_array.append(content_cleaned)

content_cleaned = re.sub(r'ADVERTISEMENT', '', teks2)
content_cleaned = re.sub(r'KOMPAS.com', '', content_cleaned)
content_cleaned = re.sub(r'[^a-zA-Z0-9\s]+', ' ', content_cleaned)
teks_array.append(content_cleaned)

content_cleaned = re.sub(r'ADVERTISEMENT', '', teks3)
content_cleaned = re.sub(r'KOMPAS.com', '', content_cleaned)
content_cleaned = re.sub(r'[^a-zA-Z0-9\s]+', ' ', content_cleaned)
teks_array.append(content_cleaned)

print(teks_array)

[' Panglima TNI Laksamana Yudo Margono buka suara soal sosok KSAD baru Jenderal TNI Agus Subiyanto yang disodorkan Presiden RI untuk dites DPR sebagai calon penggantinya Yudo yang akan pensiun itu meyakini kemampuan Agus seandainya resmi memimpin TNI sebagai pengganti dirinya kelak Pak Agus Subiyanto baik  Saya yakin beliau mampu untuk memimpin TNI  tentunya nanti bersinergi dengan Polri dan juga bisa bersama sama kepala staf yang lain untuk membawa TNI yang lebih baik  lebih profesional  modern  tangguh  kata Yudo di Sesko TNI  Bandung  Kamis  2 11 Yudo menuturkan saat dirinya dilantik  ada beberapa agenda kegiatan besar TNI yang telah disusunnya sebagai panglima  Salah satunya adalah mengawal proses berjalan Pemilu 2024  Itu  kata dia  akan menjadi pijakan juga bagi panglima pengganti dirinya Dalam perencanaan kegiatan sudah ada semuanya  yang kita susun di tahun 2023 ini dan nantinya 2024 apa saja  tentunya Pemilu  berikutnya dalam operasi latihan  operasi juga tetap sekarang yang a

In [3]:
teks_array_lowercase = []

for teks in teks_array:
    teks_lowercase = teks.lower()
    teks_array_lowercase.append(teks_lowercase)

print(teks_array_lowercase)

[' panglima tni laksamana yudo margono buka suara soal sosok ksad baru jenderal tni agus subiyanto yang disodorkan presiden ri untuk dites dpr sebagai calon penggantinya yudo yang akan pensiun itu meyakini kemampuan agus seandainya resmi memimpin tni sebagai pengganti dirinya kelak pak agus subiyanto baik  saya yakin beliau mampu untuk memimpin tni  tentunya nanti bersinergi dengan polri dan juga bisa bersama sama kepala staf yang lain untuk membawa tni yang lebih baik  lebih profesional  modern  tangguh  kata yudo di sesko tni  bandung  kamis  2 11 yudo menuturkan saat dirinya dilantik  ada beberapa agenda kegiatan besar tni yang telah disusunnya sebagai panglima  salah satunya adalah mengawal proses berjalan pemilu 2024  itu  kata dia  akan menjadi pijakan juga bagi panglima pengganti dirinya dalam perencanaan kegiatan sudah ada semuanya  yang kita susun di tahun 2023 ini dan nantinya 2024 apa saja  tentunya pemilu  berikutnya dalam operasi latihan  operasi juga tetap sekarang yang a

In [4]:
teks_word=[]
for teks in teks_array_lowercase:
    teks_to = stopword_remover.remove(teks)
    teks_word.append(teks_to)
    
print(teks_word)

[' panglima tni laksamana yudo margono buka suara sosok ksad jenderal tni agus subiyanto disodorkan presiden ri dites dpr calon penggantinya yudo pensiun kemampuan agus seandainya resmi memimpin tni pengganti kelak agus subiyanto  beliau memimpin tni  bersinergi polri kepala staf membawa tni  profesional  modern  tangguh  yudo sesko tni  bandung  kamis  2 11 yudo dilantik  agenda kegiatan tni disusunnya panglima  salah satunya mengawal proses berjalan pemilu 2024    pijakan panglima pengganti perencanaan kegiatan  susun tahun 2023 2024  pemilu  operasi latihan  operasi  latihan rancang tahun 2024  yudo memasuki pensiun november 2023 dpr menerima surat presiden  surpres  jokowi menguji kepatutan kelayakan  fit and proper test  agus subiyanto calon panglima tni  komisi dpr menjadwalkan uji kepatutan kelayakan digelar 14 november yudo  disinggung langkah melepas jabatan purnawirawan  mengaku memfokuskan bertani  bertani  bertani  bertani padi  kebun   bertani pokoknya   bertaninya  rahasi

In [5]:
teks_tokenize=[]
for teks in teks_word:
    teks_to = word_tokenize(teks)
    teks_tokenize.append(teks_to)
    
print(teks_tokenize)

[['panglima', 'tni', 'laksamana', 'yudo', 'margono', 'buka', 'suara', 'sosok', 'ksad', 'jenderal', 'tni', 'agus', 'subiyanto', 'disodorkan', 'presiden', 'ri', 'dites', 'dpr', 'calon', 'penggantinya', 'yudo', 'pensiun', 'kemampuan', 'agus', 'seandainya', 'resmi', 'memimpin', 'tni', 'pengganti', 'kelak', 'agus', 'subiyanto', 'beliau', 'memimpin', 'tni', 'bersinergi', 'polri', 'kepala', 'staf', 'membawa', 'tni', 'profesional', 'modern', 'tangguh', 'yudo', 'sesko', 'tni', 'bandung', 'kamis', '2', '11', 'yudo', 'dilantik', 'agenda', 'kegiatan', 'tni', 'disusunnya', 'panglima', 'salah', 'satunya', 'mengawal', 'proses', 'berjalan', 'pemilu', '2024', 'pijakan', 'panglima', 'pengganti', 'perencanaan', 'kegiatan', 'susun', 'tahun', '2023', '2024', 'pemilu', 'operasi', 'latihan', 'operasi', 'latihan', 'rancang', 'tahun', '2024', 'yudo', 'memasuki', 'pensiun', 'november', '2023', 'dpr', 'menerima', 'surat', 'presiden', 'surpres', 'jokowi', 'menguji', 'kepatutan', 'kelayakan', 'fit', 'and', 'proper